### Import various libraries and packages

In [ ]:
from afft.utils.log import logger

### Create SQL engine
Create a SQL engine with SQLalchemy. Note that the engine has not connected to the database yet.

In [ ]:
import sqlalchemy as db

from dotenv import dotenv_values


values = dotenv_values("/home/martin/dev/afft/.env")


url: db.engine.URL = db.engine.URL.create(
    drivername="postgresql",
    database="acfr_auv_revisits",
    host="localhost",
    port=5432,
    username=values.get("PG_USER"),
    password=values.get("PG_PASSWORD"),
)

engine: db.Engine = db.create_engine(url)

### Connect to database
To start interacting with the database you can be explicit and use the connect method.

In [ ]:
from sqlalchemy.exc import OperationalError

if False:
    try:
        engine.connect()
    except (OperationalError, OSError, IOError) as error:
        logger.error(error)

### Read and parse messages

In [ ]:
from pathlib import Path

from afft.io import read_config, read_lines
from afft.services.sirius.message_interfaces import Message
from afft.services.sirius.message_set import (
    MessageSet,
    build_message_set,
    parse_message_lines,
)
from afft.utils.log import logger


def parse_messages(source: str, config: str) -> dict[str, Message]:
    """Reads message lines from a source and parses the messages."""

    config: dict = read_config(Path(config)).unwrap()
    lines: list[str] = read_lines(Path(source)).unwrap()

    message_set: MessageSet = build_message_set(config.get("message_maps"))
    parsed_messages: dict[str, Message] = parse_message_lines(lines, message_set)

    return parsed_messages


MESSAGE_DIR: Path = Path(
    "/home/martin/data/acfr_revisits_messages/acfr_merged_messages"
)
MESSAGE_FILE: Path = MESSAGE_DIR / Path("r23685bc_20100605_021022_messages.txt")
PROTOCOL_FILE: Path = Path("/home/martin/dev/afft/config/protocol/protocol_v1.toml")

parsed_messages: dict[str, Message] = parse_messages(MESSAGE_FILE, PROTOCOL_FILE)

for topic, messages in parsed_messages.items():
    logger.info(f"Topic: {topic}, message count: {len(messages)}")

for topic, messages in parsed_messages.items():
    logger.info(f"Topic: {topic}, sample: {messages[0]}")

### Create tables
To create tables, extract metadata from

In [ ]:
with engine.begin() as connection:
    metadata = db.MetaData()  # extracting the metadata

    logger.info(metadata)

    student = db.Table(
        "test",
        metadata,
        db.Column("id", db.Integer(), primary_key=True),
        db.Column("name", db.String(255), nullable=False),
        db.Column("major", db.String(255), default="Math"),
        db.Column("passed", db.Boolean(), default=True),
    )

    # Create the tables
    metadata.create_all(engine)

### Load table

In [ ]:
# division= db.Table('divisions', metadata, autoload_with=engine)
query = db.insert(student).values(id=1, name="Matthew", major="English", passed=True)
result = connection.execute(query)